In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#import tensorflow as tf #
import string #
#import requests #
import pandas as pd #


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences


# https://en.wikipedia.org/wiki/Pseudoword
# Source: https://kgptalkie.com/poetry-generation-using-tensorflow-keras-and-lstm/

In [ ]:
#response = requests.get('https://raw.githubusercontent.com/laxmimerit/poetry-data/master/adele.txt')
#response.text

In [2]:
# Load the Polish wordlist
text = str()
with open('/kaggle/input/languages-of-europe/pl_PL.csv', encoding = "iso-8859-2") as f:
    text += f.read()
# check
text[:10]    

'a\nA\naa\nAA\n'

In [3]:
# seperate all characters with space
text = " ".join(text)
# check
text[:10]  

'a \n A \n a '

In [4]:
# get initial n characters for test
text = text[:10000]
# check
text[:10]

'a \n A \n a '

In [5]:
# split text into lines at each \n
data = text.splitlines()
# check
data[:10]

['a ',
 ' A ',
 ' a a ',
 ' A A ',
 ' a a a ',
 ' A a c h e n ',
 ' A a l b o r g ',
 ' A a l t o ',
 ' A A N ',
 ' A A P ']

In [6]:
# that way we have a number of lines which should be number of rows in csv
len(data)

516

In [7]:
# check lenght of text
len(" ".join(data))

10000

In [8]:
# vectorize the list of texts
token = Tokenizer()
# update internal vocabulary based on list of texts
token.fit_on_texts(data)

In [9]:
# check word frequency in tokenizer
token.word_counts


OrderedDict([('a', 966),
             ('c', 285),
             ('h', 46),
             ('e', 223),
             ('n', 191),
             ('l', 169),
             ('b', 437),
             ('o', 261),
             ('r', 185),
             ('g', 33),
             ('t', 226),
             ('p', 14),
             ('d', 85),
             ('v', 23),
             ('k', 118),
             ('u', 143),
             ('s', 207),
             ('i', 225),
             ('j', 105),
             ('ń', 11),
             ('z', 89),
             ('y', 199),
             ('w', 101),
             ('ż', 18),
             ('m', 66),
             ('ć', 19),
             ('ą', 17),
             ('ł', 2),
             ('f', 8),
             ('ó', 4),
             ('x', 2),
             ('ś', 7)])

In [10]:
# check word indices in tokenizer
token.word_index

{'a': 1,
 'b': 2,
 'c': 3,
 'o': 4,
 't': 5,
 'i': 6,
 'e': 7,
 's': 8,
 'y': 9,
 'n': 10,
 'r': 11,
 'l': 12,
 'u': 13,
 'k': 14,
 'j': 15,
 'w': 16,
 'z': 17,
 'd': 18,
 'm': 19,
 'h': 20,
 'g': 21,
 'v': 22,
 'ć': 23,
 'ż': 24,
 'ą': 25,
 'p': 26,
 'ń': 27,
 'f': 28,
 'ś': 29,
 'ó': 30,
 'ł': 31,
 'x': 32}

In [11]:
# set size of the vocabulary we will need it for hot encoding target variable
vocab_size = len(token.word_counts) + 1
vocab_size

33

In [12]:
# encode our texts into integers with word indices
encoded_text = token.texts_to_sequences(data)
#check
encoded_text[:10]

[[1],
 [1],
 [1, 1],
 [1, 1],
 [1, 1, 1],
 [1, 1, 3, 20, 7, 10],
 [1, 1, 12, 2, 4, 11, 21],
 [1, 1, 12, 5, 4],
 [1, 1, 10],
 [1, 1, 26]]

In [13]:
# check how any text is converted to integers
x = ['j a c e k']
token.texts_to_sequences(x)

[[15, 1, 3, 7, 14]]

In [17]:
# Prepare Training Data
# Filter out words of len >1
# From each word d make left sub words of lenght 2, 3, ..., len(d) - 1
datalist = []
for d in encoded_text:
  if len(d)> 1:
    for i in range(2, len(d)): # ? why we skip the last character?
      datalist.append(d[:i])
#      print(d[:i])

#check
datalist[:10]

[[1, 1],
 [1, 1],
 [1, 1, 3],
 [1, 1, 3, 20],
 [1, 1, 3, 20, 7],
 [1, 1],
 [1, 1, 12],
 [1, 1, 12, 2],
 [1, 1, 12, 2, 4],
 [1, 1, 12, 2, 4, 11]]

In [15]:
datalist = []
d = encoded_text[5]
print(d)
len(d)
if len(d)> 1:
    for i in range(2, len(d)+1): 
      datalist.append(d[:i])
#      print(d[:i])
datalist

[1, 1, 3, 20, 7, 10]


[[1, 1], [1, 1, 3], [1, 1, 3, 20], [1, 1, 3, 20, 7], [1, 1, 3, 20, 7, 10]]

My name  = 'j a c e k' is encoded into integer '15, 1, 3, 7, 14'

Then we make 

'15, 1'
'15, 1, 3'
'15, 1, 3, 7'

Then we make padding

'0, 0, 0,15, 1'
'0, 0,15, 1, 3'
'0,15, 1, 3, 7'




In [18]:
# the new list is naturaly much longer
len(datalist)

3455

In [19]:
# transform our list into 2D Numpy array of shape len(datalist) * max_length (Padding)
max_length = 20
max_length = 10
sequences = pad_sequences(datalist, maxlen=max_length, padding='pre')
#check
sequences

array([[ 0,  0,  0, ...,  0,  1,  1],
       [ 0,  0,  0, ...,  0,  1,  1],
       [ 0,  0,  0, ...,  1,  1,  3],
       ...,
       [ 0,  0,  1, ...,  4, 16,  6],
       [ 0,  0,  0, ...,  0,  1,  3],
       [ 0,  0,  0, ...,  1,  3, 20]], dtype=int32)

In [20]:
# check the hape of the result
sequences.shape

(3455, 10)

we have only trainong data set
columns without the last are input variables
last column is target for classification algorithm

In [21]:
# abservations and features for classification
X = sequences[:, :-1]
# check
X[:10]

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  0,  0,  0,  1,  1],
       [ 0,  0,  0,  0,  0,  0,  1,  1,  3],
       [ 0,  0,  0,  0,  0,  1,  1,  3, 20],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  0,  0,  0,  1,  1],
       [ 0,  0,  0,  0,  0,  0,  1,  1, 12],
       [ 0,  0,  0,  0,  0,  1,  1, 12,  2],
       [ 0,  0,  0,  0,  1,  1, 12,  2,  4]], dtype=int32)

In [22]:
# target variable converted to factor
y = sequences[:, -1]
y = to_categorical(y, num_classes=vocab_size)
# character in target is hot encoded
# check
y[:10]

array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.

In [23]:
# this the number of features  we will need it ? 
# we solve  multiclass classification problem
seq_length = X.shape[1]
seq_length

9

In [24]:
# We initiate stack with sequential layer
model = Sequential()
# We turn positive integers (indexes) into dense vectors of fixed size
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))


In [25]:
# check the added layers
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 9, 50)             1650      
_________________________________________________________________
lstm (LSTM)                  (None, 9, 100)            60400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 33)                3333      
Total params: 155,883
Trainable params: 155,883
Non-trainable params: 0
_________________________________________________________________


In [29]:
# We configure the model for training
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# We train the model for a fixed number of epochs (iterations on a dataset)
model.fit(X, y, batch_size=32, epochs=50)

Epoch 1/50
108/108 [==============================] - 6s 22ms/step - loss: 3.1784 - accuracy: 0.1272
Epoch 2/50
108/108 [==============================] - 2s 22ms/step - loss: 2.6543 - accuracy: 0.2050
Epoch 3/50
108/108 [==============================] - 2s 21ms/step - loss: 2.5911 - accuracy: 0.2049
Epoch 4/50
108/108 [==============================] - 2s 20ms/step - loss: 2.5525 - accuracy: 0.2215
Epoch 5/50
108/108 [==============================] - 2s 20ms/step - loss: 2.4896 - accuracy: 0.2589
Epoch 6/50
108/108 [==============================] - 2s 20ms/step - loss: 2.4807 - accuracy: 0.2539
Epoch 7/50
108/108 [==============================] - 2s 19ms/step - loss: 2.4049 - accuracy: 0.2730
Epoch 8/50
108/108 [==============================] - 2s 19ms/step - loss: 2.3325 - accuracy: 0.2985
Epoch 9/50
108/108 [==============================] - 2s 19ms/step - loss: 2.3052 - accuracy: 0.3110
Epoch 10/50
108/108 [==============================] - 2s 21ms/step - loss: 2.2290 - accura

In [148]:
poetry_length = 5 # lenght of the returned word '_', number of repetitions
n_lines = 2 # number of words 'i'
test_word = 'awo' # input word # podaj pierwszą literę , początek pseudo word
test_word = " ".join(test_word) # add spaces

text = []
for _ in range(poetry_length):
    encoded = token.texts_to_sequences([test_word]) # encode into integer vector
    encoded = pad_sequences(encoded, maxlen=seq_length, padding='pre') # apply padding so we have as long as number features
#encoded
    pred = model.predict(encoded) # make prediction, returns probabilities of the next word from dictionary
    y_pred = np.argmax(pred, axis=-1)[0] # take the highest probability and return
#y_pred
#predicted_word = ""
    for word, index in token.word_index.items():
        if index == y_pred:
            predicted_word = word
    test_word = test_word + ' ' + predicted_word
#text.append(predicted_word
test_word

'a w o r d a ż o'

In [225]:
# start defines start of the pseudoword, one or more chars
# length defines how many chars to add
def pseudoword(start, length):
    temp = " ".join(start) # add spaces
    for _ in range(length):
        encoded = token.texts_to_sequences([temp]) # encode into integer vector
        encoded = pad_sequences(encoded, maxlen=seq_length, padding='pre') # apply padding so we have as long as number features
        pred = model.predict(encoded) # make prediction, returns probabilities of the next word from dictionary
        y_pred = np.argmax(pred, axis=-1)[0] # take the highest probability and return
        predicted_word = ""
        for word, index in token.word_index.items():
            if index == y_pred:
                predicted_word = word
        temp = temp + ' ' + predicted_word
    temp = temp.replace(" ", "")
    print(temp)
    
 

In [231]:
pseudoword(start = "pok", length = 7)  

pokcentiat
